In [1]:
import os.path as osp
from tqdm.auto import tqdm
import numpy as np
import wandb

import torch
from sklearn.metrics import roc_auc_score

import torch.nn.functional as F
# from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
from torch.nn import LayerNorm, Linear, ReLU, Tanh
from torchvision.ops import MLP
from torchvision.utils import save_image


import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GatedGraphConv, SAGEConv, GAT
from torch_geometric.nn import DeepGCNLayer, GENConv
from torch_geometric.utils import negative_sampling, to_dense_adj
from torch_geometric.loader import DataLoader

from dataset_processing6 import RNADataset


/home/vdshk/SecondaryStructurePredictionGNN/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = RNADataset(root="./data6/")
dataset = dataset.shuffle()
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]
bs = 1

train_dataloader = DataLoader(train_data, batch_size=bs, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)

In [3]:
dataset = list(filter(lambda x: x.s.item() <= 150, dataset))

[Data(x=[2739, 6], edge_index=[2, 270034], y=[2739], s=120),
 Data(x=[42, 6], edge_index=[2, 462], y=[42], s=13),
 Data(x=[2024, 6], edge_index=[2, 171532], y=[2024], s=102),
 Data(x=[188, 6], edge_index=[2, 4610], y=[188], s=30),
 Data(x=[2774, 6], edge_index=[2, 275042], y=[2774], s=120),
 Data(x=[376, 6], edge_index=[2, 13358], y=[376], s=43),
 Data(x=[51, 6], edge_index=[2, 630], y=[51], s=15),
 Data(x=[3360, 6], edge_index=[2, 370088], y=[3360], s=133),
 Data(x=[268, 6], edge_index=[2, 8170], y=[268], s=35),
 Data(x=[308, 6], edge_index=[2, 9838], y=[308], s=40),
 Data(x=[98, 6], edge_index=[2, 1644], y=[98], s=23),
 Data(x=[262, 6], edge_index=[2, 7960], y=[262], s=34),
 Data(x=[78, 6], edge_index=[2, 1200], y=[78], s=19),
 Data(x=[152, 6], edge_index=[2, 3308], y=[152], s=29),
 Data(x=[2795, 6], edge_index=[2, 278256], y=[2795], s=120),
 Data(x=[357, 6], edge_index=[2, 12448], y=[357], s=43),
 Data(x=[887, 6], edge_index=[2, 51058], y=[887], s=66),
 Data(x=[3153, 6], edge_index=

In [4]:
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]

In [5]:
def del_nucl_conn(y_pr, y_tr):    
    y_tr_local = torch.clone(y_tr)
    y_tr_local[(y_tr_local == 1)] = 0
    y_pr = y_pr + y_tr_local
    y_pr[(y_pr < 0)] = 0
    y_tr_local1 = torch.clone(y_tr)
    y_tr_local1[(y_tr_local1 == -1)] = 0
    return y_pr, y_tr_local1


def del_main_diag(y):
    main_diag = torch.ones([1, y.size()[0]])
    return y * (torch.diag_embed(main_diag).to(device) - 1)[0] * (-1)


def adj_mat_split(y):
    y = list(torch.split(y, 196, dim=0))
    for i in range(len(y)):
        y[i] = torch.split(y[i], 196, dim=1)
    
    y1 = []
    for i in range(len(y)):
        y1.append(argm(y[i][i]))
    return torch.cat(y1, 0).to(device)
    

def argm(y):
    max_ind = torch.argmax(y, dim=1)
    y1 = torch.zeros_like(y)
    k = 0
    for i in max_ind:
        y1[k][i] = 1.
        y1[i][k] = 1.
        k += 1
    
    return y1 * y


In [6]:
a = 0.05
beta = 1
def precision(y_pred, y_true):
    y_pred, y_true = del_nucl_conn(y_pred, y_true)
    y_pred[(y_pred > a)] = 1
    y_pred[(y_pred <= a)] = 0 
#     y_pred = argm(y_pred)
    tp = torch.sum(y_pred * y_true)
    fp = torch.sum((1 - y_true) * y_pred)
    
    return tp / (tp + fp + epsilon)

def recall(y_pred, y_true):
    y_pred, y_true = del_nucl_conn(y_pred, y_true)
    y_pred[(y_pred > a)] = 1
    y_pred[(y_pred <= a)] = 0 
#     y_pred = argm(y_pred)
#     y_pred = adj_mat_split(y_pred)

    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    
    return tp / (tp + fn + epsilon)

def f_loss(y_pred, y_true):
    y_pred, y_true = del_nucl_conn(y_pred, y_true)
    
    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    fp = torch.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    k1 = 1 - torch.abs(precision - recall)

    f1 = ((1 + beta * beta) * precision * recall) / (beta * beta * precision + recall + epsilon)
    
    
    dp = abs(torch.mean(y_pred) - torch.mean(y_true))

    return (1 - f1)
#     return dp

def mse_loss(y_pred, y_true):
    y_pred, y_true = del_nucl_conn(y_pred, y_true)
    
    return torch.mean((y_pred - y_true) ** 2)

In [10]:
class DeepGCNModel(torch.nn.Module):
    def __init__(self, num_blocks, hidden_channels, num_layers, dr, mlp):
        super(DeepGCNModel, self).__init__()
        self.layers = torch.nn.ModuleList()
        gat1 = GAT(6, hidden_channels, num_layers=num_layers)
        layer = DeepGCNLayer(gat1, block='res', act=torch.tanh, dropout=dr)
        self.layers.append(layer)
        for i in range(1, num_blocks):
            conv = GatedGraphConv(hidden_channels, num_layers=num_layers, act='tanh')            

            layer = DeepGCNLayer(conv, block='res', act=torch.tanh, dropout=dr)
            self.layers.append(layer)
            
            
        self.mlp = MLP(hidden_channels, mlp, dropout=dr)

        
    def forward(self, x, edge_index):
        x = self.layers[0].conv(x, edge_index)
        for layer in self.layers[1:]:
            x = layer(x, edge_index)
        
        x = self.mlp(x)
        
        return x.sigmoid()


In [19]:
num_blocks = 1
num_layers = 2
hidden_channels = 250

dr = 0.2


lr = 0.0001
epochs = 1500
mlp = [200, 150, 100, 50, 10, 1]


model = DeepGCNModel(num_blocks, hidden_channels, num_layers, dr,mlp)
# model = torch.load("./models1/ResGATConv__MSE_mlp_[200, 150, 100, 50, 10, 1]_1*2_0.0001_Adam_250_0.05_0.2_1_1500.pt")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_data = list(map(lambda x: x.to(device), train_data))
val_data = list(map(lambda x: x.to(device), val_data))
print(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
# optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
#optimizer = torch.optim.Adamax(model.parameters(), lr=lr)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=lr)

# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# criterion = torch.nn.BCELoss()
# criterion = torch.nn.BCEWithLogitsLoss()
# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.KLDivLoss()

def RMSELoss(y_pred, y_true):
    y_pred, y_true = del_nucl_conn(y_pred, y_true)

    return torch.sqrt(torch.sum((y_pred - y_true) ** 2))

criterion = mse_loss
# criterion = RMSELoss

epsilon = 1e-10

cuda


In [20]:
model

DeepGCNModel(
  (layers): ModuleList(
    (0): DeepGCNLayer(block=res)
  )
  (mlp): MLP(
    (0): Linear(in_features=250, out_features=200, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=200, out_features=150, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=150, out_features=100, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=100, out_features=50, bias=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
    (12): Linear(in_features=50, out_features=10, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.2, inplace=False)
    (15): Linear(in_features=10, out_features=1, bias=True)
    (16): Dropout(p=0.2, inplace=False)
  )
)

In [21]:
run_name = f'ResGATConv__MSE_mlp_{mlp}_{num_blocks}*{num_layers}_{lr}_Adam_{hidden_channels}_{a}_{dr}_{beta}'


wandb.init(
    # set the wandb project where this run will be logged
    project="secondary_structure_prediction1",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "ResGATConv",
    "epochs": epochs,
    "optimizer": "Adam",
    "out_channels": hidden_channels,
    "loss": "MSE",
    "beta": beta,
    "train:val:test": "655:218:218",
    "dataset": 6
    },
    name=run_name
)



train_f1,▁▁▃█▇█▇█████████████████████████████████
train_loss,█▇▅▃▃▃▃▃▂▂▂▂▂▂▂▁▂▂▁▂▂▂▂▁▁▂▁▂▂▁▁▂▁▁▁▂▁▁▁▁
train_precision,▁▁▃█▇█▇██████▇██████████████████████████
train_recall,▁▂▃▆▆▇▇▇▇▇▇██▇██████▇█▇█████████████████
val_f1,▁▁▃▇▇█▇▇▇▇▇▇▇▇▇▇████▇▇▇█▇▇▇▇██▇▇██▇████▇
val_loss,█▅▆▃▃▂▂▃▃▃▃▃▂▃▃▃▃▂▂▂▃▃▂▂▃▄▃▄▃▂▃▂▂▁▃▄▃▃▃▄
val_precision,▁▁▃█▇█▇▇▇▇▇▇▇▇▇▇███▇▇▇▇█▇▇▇▇██▇▇██▇████▇
val_recall,▁▂▄▆▇▇▇▇▇▇▇▇▇▇██▇██▇▇██▇▇█████▇█▇▇▇████▇
train_f1,0.13358
train_loss,0.12203
train_precision,0.07234


In [22]:
def train():
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        train_recall = []
        train_precision = []
        for g in tqdm(train_data):

            optimizer.zero_grad()

            out = model(g.x, g.edge_index)
            y_true = g.y.unsqueeze(dim=1)
            loss = criterion(out, y_true)            
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            train_precision.append(precision(out, y_true).item())
            train_recall.append(recall(out, y_true).item())
            
        train_prec = np.mean(train_precision)
        train_rec = np.mean(train_recall)
        train_f1 = (2 * train_prec * train_rec) / (train_prec + train_rec)
        print(f'Epoch: {epoch:03d}, loss: {np.mean(train_loss)}, f1: {train_f1}, precision: {train_prec}, recall: {train_rec}')
        
        val_loss = []
        val_recall = []
        val_precision = []
        with torch.no_grad():
            for g in val_data:
                g.to(device)
                out = model(g.x, g.edge_index)
        
                y_true = g.y.unsqueeze(dim=1)
                loss = criterion(out, y_true)
                                
                val_loss.append(loss.item())
                                
                val_precision.append(precision(out, y_true).item())
                val_recall.append(recall(out, y_true).item())
 
            
            prec = np.mean(val_precision)
            rec = np.mean(val_recall)
            f1 = (2 * prec * rec) / (prec + rec)
            print(f'val_loss: {np.mean(val_loss)}, val_f1: {f1}, val_precision: {prec}, val_recall: {rec}')
              
            wandb.log({"train_loss": np.mean(train_loss), "train_f1": train_f1, "train_precision": train_prec, 
                       "train_recall": train_rec,
                       "val_loss": np.mean(val_loss), "val_f1": f1, "val_precision": prec, "val_recall": rec})
#     wandb.finish()

In [23]:
train()
torch.save(model, "./models1/" + run_name + "_" + str(epochs) + ".pt")

100%|█████████████████████████████████████████| 655/655 [00:14<00:00, 46.30it/s]


Epoch: 001, loss: 0.08189552787492294, f1: 0.08489153291334466, precision: 0.04748974511762441, recall: 0.39963266418123516
val_loss: 0.07341126829317403, val_f1: 0.08544279616043142, val_precision: 0.049341584720281426, val_recall: 0.31841118647417893


100%|█████████████████████████████████████████| 655/655 [00:14<00:00, 46.78it/s]


Epoch: 002, loss: 0.07319675129447274, f1: 0.08400715714276596, precision: 0.048014740657615865, recall: 0.33550766105870256
val_loss: 0.07408668104251591, val_f1: 0.08016442826818661, val_precision: 0.046381296383840705, val_recall: 0.29513268437612494


100%|█████████████████████████████████████████| 655/655 [00:13<00:00, 46.92it/s]


...

Epoch: 1498, loss: 0.059616695374307284, f1: 0.21240253606488096, precision: 0.12332272008914529, recall: 0.7649485106345352
val_loss: 0.068895236990594, val_f1: 0.20474918618327853, val_precision: 0.11918489808652448, val_recall: 0.7258348642686091


100%|█████████████████████████████████████████| 655/655 [00:13<00:00, 47.35it/s]


Epoch: 1499, loss: 0.06024311934775524, f1: 0.21339907658849647, precision: 0.12383996889364855, recall: 0.7709063915692212
val_loss: 0.06816805624018568, val_f1: 0.20302309550577585, val_precision: 0.11756545358439993, val_recall: 0.7433861442115328


100%|█████████████████████████████████████████| 655/655 [00:13<00:00, 47.33it/s]


Epoch: 1500, loss: 0.06016344536124295, f1: 0.2130374173902152, precision: 0.12360041892148896, recall: 0.7707516895678207
val_loss: 0.06796413291365318, val_f1: 0.20497388759265364, val_precision: 0.11900188839661145, val_recall: 0.7384911147416184


In [ ]:
torch.save(model, "./models1/" + run_name + "_" + str(22) + ".pt")

In [87]:
wandb.finish()